In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 3D 시각화를 위한 라이브러리
from mpl_toolkits.mplot3d import Axes3D

# 모델 요약을 위한 라이브러리
from torchsummary import summary

# 데이터셋과 데이터로더 생성
from torch.utils.data import DataLoader, TensorDataset


In [ ]:

# 데이터 생성
np.random.seed(0)
torch.manual_seed(0)

# 입력 데이터 생성
x1 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
x2 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
X1, X2 = np.meshgrid(x1, x2)

# 입력 데이터를 벡터 형태로 변환
X_input = np.stack([X1.flatten(), X2.flatten()], axis=1)

# 실제 함수 값 계산
def f_true(x):
    return np.sin(x[:, 0] + x[:, 1])

Y_true = f_true(X_input)

# 데이터를 PyTorch 텐서로 변환
X_input_tensor = torch.from_numpy(X_input).float()
Y_true_tensor = torch.from_numpy(Y_true).float()

In [ ]:
batch_size = 1024
dataset = TensorDataset(X_input_tensor, Y_true_tensor)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Kolmogorov-Arnold Network 정의
class KAN(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(KAN, self).__init__()
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        # 각 입력 차원에 대한 일변량 함수 φ_i
        self.phi_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(1, hidden_size),
                nn.Tanh(),
                nn.Linear(hidden_size, hidden_size),
                nn.Tanh()
            ) for _ in range(input_dim)
        ])
        # 최종 일변량 함수 ψ
        self.psi = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
    def forward(self, x):
        # x: [batch_size, input_dim]
        outputs = []
        for i in range(self.input_dim):
            xi = x[:, i].unsqueeze(1)  # [batch_size, 1]
            phi_i = self.phi_layers[i](xi)  # [batch_size, hidden_size]
            outputs.append(phi_i)
        # 출력 합산
        s = sum(outputs)  # [batch_size, hidden_size]
        # 최종 함수 적용
        out = self.psi(s)  # [batch_size, 1]
        return out.squeeze()

In [ ]:
# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze()

In [ ]:
# Saver 클래스 정의
class Saver:
    def __init__(self):
        self.outputs = []

    def save(self, epoch, Y_pred):
        self.outputs.append((epoch, Y_pred))

In [ ]:
# 모델 초기화
input_dim = 2
hidden_size = 10

kan_model = KAN(input_dim=input_dim, hidden_size=hidden_size)
mlp_model = MLP(input_dim=input_dim, hidden_size=hidden_size*20) # mlp model의 파라미터가 100배 이상 많도록

# 모델 요약 출력
print("KAN 모델 요약:")
summary(kan_model, input_size=(input_dim,), device='cpu')

print("\nMLP 모델 요약:")
summary(mlp_model, input_size=(input_dim,), device='cpu')


In [ ]:
# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
kan_optimizer = optim.Adam(kan_model.parameters(), lr=0.001)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# 학습 루프
epochs = 100

# 에포크별 손실 저장용 리스트
kan_losses = []
mlp_losses = []

# 시각화를 위한 예측값 저장용 Saver 객체 생성
kan_saver = Saver()
mlp_saver = Saver()

# 시각화할 에포크 지정 (5개의 에포크로 줄임)
visualization_epochs = [19, 39, 59, 79, 99]  # 총 5개의 에포크

for epoch in range(epochs):
    kan_model.train()
    mlp_model.train()
    
    kan_epoch_loss = 0
    mlp_epoch_loss = 0
    
    for X_batch, Y_batch in loader:
        # 그래디언트 초기화
        kan_optimizer.zero_grad()
        mlp_optimizer.zero_grad()
        
        # KAN 모델 학습
        kan_outputs = kan_model(X_batch)
        kan_loss = criterion(kan_outputs, Y_batch)
        kan_loss.backward()
        kan_optimizer.step()
        
        kan_epoch_loss += kan_loss.item() * X_batch.size(0)
        
        # MLP 모델 학습
        mlp_outputs = mlp_model(X_batch)
        mlp_loss = criterion(mlp_outputs, Y_batch)
        mlp_loss.backward()
        mlp_optimizer.step()
        
        mlp_epoch_loss += mlp_loss.item() * X_batch.size(0)
    
    # 에포크별 평균 손실 계산
    kan_epoch_loss /= len(dataset)
    mlp_epoch_loss /= len(dataset)
    
    # 손실 저장
    kan_losses.append(kan_epoch_loss)
    mlp_losses.append(mlp_epoch_loss)
    
    # 손실 출력
    print(f'Epoch {epoch+1}/{epochs}, KAN Loss: {kan_epoch_loss:.4f}, MLP Loss: {mlp_epoch_loss:.4f}')
    
    # 특정 에포크마다 예측값 저장
    if epoch in visualization_epochs:
        # 모델 평가 모드로 전환
        kan_model.eval()
        mlp_model.eval()
        
        with torch.no_grad():
            kan_pred = kan_model(X_input_tensor)
            mlp_pred = mlp_model(X_input_tensor)
        
        # 예측 값 저장
        kan_saver.save(epoch, kan_pred.numpy())
        mlp_saver.save(epoch, mlp_pred.numpy())


In [ ]:
# 실제 함수 값
Z_true = Y_true.reshape(X1.shape)

# 손실 곡선 시각화
plt.figure(figsize=(10,5))
plt.plot(range(1, epochs+1), kan_losses, label='KAN Loss')
plt.plot(range(1, epochs+1), mlp_losses, label='MLP Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

In [ ]:
# 에포크별 결과 시각화 함수 정의
def plot_approximations(X1, X2, Y_true, kan_saver, mlp_saver):
    num_cols = 5  # 한 행에 보여줄 그래프 수
    num_rows = 3  # 실제 함수 + KAN + MLP

    fig = plt.figure(figsize=(4*num_cols, 4*num_rows))

    # 실제 함수 시각화 (첫 번째 행)
    for i in range(num_cols):
        ax = fig.add_subplot(num_rows, num_cols, i+1, projection='3d')
        if i == 0:
            ax.plot_surface(X1, X2, Y_true.reshape(X1.shape), cmap='viridis')
            ax.set_title('Actual Function')
            ax.set_xlabel('x1')
            ax.set_ylabel('x2')
            ax.set_zlabel('f(x1, x2)')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_zticks([])
        else:
            # 빈 플롯 생성
            ax.axis('off')

    # KAN 에포크별 근사 결과 시각화 (두 번째 행)
    for i, (epoch, Y_pred) in enumerate(kan_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'KAN Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    # MLP 에포크별 근사 결과 시각화 (세 번째 행)
    for i, (epoch, Y_pred) in enumerate(mlp_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, 2*num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'MLP Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    plt.tight_layout()
    plt.show()

# 전체 결과 시각화
plot_approximations(X1, X2, Y_true, kan_saver, mlp_saver)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 3D 시각화를 위한 라이브러리
from mpl_toolkits.mplot3d import Axes3D

# 모델 요약을 위한 라이브러리
from torchsummary import summary

# 데이터 생성
np.random.seed(0)
torch.manual_seed(0)

# 입력 데이터 생성
x1 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
x2 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
X1, X2 = np.meshgrid(x1, x2)

# 입력 데이터를 벡터 형태로 변환
X_input = np.stack([X1.flatten(), X2.flatten()], axis=1)

# 실제 함수 값 계산
def f_true(x):
    return np.sin(x[:, 0] + x[:, 1])

Y_true = f_true(X_input)

# 데이터를 PyTorch 텐서로 변환
X_input_tensor = torch.from_numpy(X_input).float()
Y_true_tensor = torch.from_numpy(Y_true).float()

# 데이터셋과 데이터로더 생성
from torch.utils.data import DataLoader, TensorDataset

batch_size = 1000
dataset = TensorDataset(X_input_tensor, Y_true_tensor)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# KAN 모델 정의
class KAN(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(KAN, self).__init__()
        self.input_dim = input_dim
        self.hidden_size = hidden_size

        # 각 입력 차원에 대한 일변량 함수 φ_i
        self.phi_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(1, hidden_size),
                nn.SiLU(),
                nn.Linear(hidden_size, hidden_size),
                nn.SiLU()
            ) for _ in range(input_dim)
        ])

        # 최종 일변량 함수 ψ
        self.psi = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.SiLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        # x: [batch_size, input_dim]
        outputs = []
        for i in range(self.input_dim):
            xi = x[:, i].unsqueeze(1)  # [batch_size, 1]
            phi_i = self.phi_layers[i](xi)  # [batch_size, hidden_size]
            outputs.append(phi_i)
        # 출력 합산
        s = sum(outputs)  # [batch_size, hidden_size]
        # 최종 함수 적용
        out = self.psi(s)  # [batch_size, 1]
        return out.squeeze()

# MLP 모델 정의 (KAN과 구조를 맞춤)
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_size, n_hidden_layers=3):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_size))
        layers.append(nn.SiLU())
        for _ in range(n_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.SiLU())
        layers.append(nn.Linear(hidden_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze()

# Saver 클래스 정의
class Saver:
    def __init__(self):
        self.outputs = []

    def save(self, epoch, Y_pred):
        self.outputs.append((epoch, Y_pred))

# 모델 초기화
input_dim = 2
hidden_size = 64  # 히든 사이즈를 늘려서 복잡한 함수 근사에 적합하게 조정
n_hidden_layers = 3  # 히든 레이어 수를 늘림

kan_model = KAN(input_dim=input_dim, hidden_size=hidden_size)
mlp_model = MLP(input_dim=input_dim, hidden_size=hidden_size*10, n_hidden_layers=n_hidden_layers)

# 모델 요약 출력
print("KAN 모델 요약:")
summary(kan_model, input_size=(input_dim,), device='cpu')

print("\nMLP 모델 요약:")
summary(mlp_model, input_size=(input_dim,), device='cpu')

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
kan_optimizer = optim.Adam(kan_model.parameters(), lr=0.001)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# 학습 루프
epochs = 100

# 에포크별 손실 저장용 리스트
kan_losses = []
mlp_losses = []

# 시각화를 위한 예측값 저장용 Saver 객체 생성
kan_saver = Saver()
mlp_saver = Saver()

# 시각화할 에포크 지정 (5개의 에포크로 줄임)
visualization_epochs = [19, 39, 59, 79, 99]  # 총 5개의 에포크

for epoch in range(epochs):
    kan_model.train()
    mlp_model.train()
    
    kan_epoch_loss = 0
    mlp_epoch_loss = 0
    
    for X_batch, Y_batch in loader:
        # 그래디언트 초기화
        kan_optimizer.zero_grad()
        mlp_optimizer.zero_grad()
        
        # KAN 모델 학습
        kan_outputs = kan_model(X_batch)
        kan_loss = criterion(kan_outputs, Y_batch)
        kan_loss.backward()
        kan_optimizer.step()
        
        kan_epoch_loss += kan_loss.item() * X_batch.size(0)
        
        # MLP 모델 학습
        mlp_outputs = mlp_model(X_batch)
        mlp_loss = criterion(mlp_outputs, Y_batch)
        mlp_loss.backward()
        mlp_optimizer.step()
        
        mlp_epoch_loss += mlp_loss.item() * X_batch.size(0)
    
    # 에포크별 평균 손실 계산
    kan_epoch_loss /= len(dataset)
    mlp_epoch_loss /= len(dataset)
    
    # 손실 저장
    kan_losses.append(kan_epoch_loss)
    mlp_losses.append(mlp_epoch_loss)
    
    # 손실 출력
    print(f'Epoch {epoch+1}/{epochs}, KAN Loss: {kan_epoch_loss:.4f}, MLP Loss: {mlp_epoch_loss:.4f}')
    
    # 특정 에포크마다 예측값 저장
    if epoch in visualization_epochs:
        # 모델 평가 모드로 전환
        kan_model.eval()
        mlp_model.eval()
        
        with torch.no_grad():
            kan_pred = kan_model(X_input_tensor)
            mlp_pred = mlp_model(X_input_tensor)
        
        # 예측 값 저장
        kan_saver.save(epoch, kan_pred.numpy())
        mlp_saver.save(epoch, mlp_pred.numpy())

# 실제 함수 값
Z_true = Y_true.reshape(X1.shape)

# 손실 곡선 시각화
plt.figure(figsize=(10,5))
plt.plot(range(1, epochs+1), kan_losses, label='KAN Loss')
plt.plot(range(1, epochs+1), mlp_losses, label='MLP Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

# 에포크별 결과 시각화 함수 정의
def plot_approximations(X1, X2, Y_true, kan_saver, mlp_saver):
    num_cols = 5  # 한 행에 보여줄 그래프 수
    num_rows = 3  # 실제 함수 + KAN + MLP

    fig = plt.figure(figsize=(4*num_cols, 4*num_rows))

    # 실제 함수 시각화 (첫 번째 행)
    for i in range(num_cols):
        ax = fig.add_subplot(num_rows, num_cols, i+1, projection='3d')
        if i == 0:
            ax.plot_surface(X1, X2, Y_true.reshape(X1.shape), cmap='viridis')
            ax.set_title('Actual Function')
            ax.set_xlabel('x1')
            ax.set_ylabel('x2')
            ax.set_zlabel('f(x1, x2)')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_zticks([])
        else:
            # 빈 플롯 생성
            ax.axis('off')

    # KAN 에포크별 근사 결과 시각화 (두 번째 행)
    for i, (epoch, Y_pred) in enumerate(kan_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'KAN Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    # MLP 에포크별 근사 결과 시각화 (세 번째 행)
    for i, (epoch, Y_pred) in enumerate(mlp_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, 2*num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'MLP Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    plt.tight_layout()
    plt.show()

# 전체 결과 시각화
plot_approximations(X1, X2, Y_true, kan_saver, mlp_saver)


KAN 모델 요약:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             128
              SiLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              SiLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]             128
              SiLU-6                   [-1, 64]               0
            Linear-7                   [-1, 64]           4,160
              SiLU-8                   [-1, 64]               0
            Linear-9                   [-1, 64]           4,160
             SiLU-10                   [-1, 64]               0
           Linear-11                    [-1, 1]              65
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 3D 시각화를 위한 라이브러리
from mpl_toolkits.mplot3d import Axes3D

# 모델 요약을 위한 라이브러리
from torchsummary import summary

# CUDA 사용 여부 확인 및 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 데이터 생성
np.random.seed(0)
torch.manual_seed(0)

# 입력 데이터 생성
x1 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
x2 = np.linspace(-2 * np.pi, 2 * np.pi, 200)
X1, X2 = np.meshgrid(x1, x2)

# 입력 데이터를 벡터 형태로 변환
X_input = np.stack([X1.flatten(), X2.flatten()], axis=1)

# 실제 함수 값 계산
def f_true(x):
    return np.sin(x[:, 0] + x[:, 1])

Y_true = f_true(X_input)

# 데이터를 PyTorch 텐서로 변환하고 장치로 이동
X_input_tensor = torch.from_numpy(X_input).float().to(device)
Y_true_tensor = torch.from_numpy(Y_true).float().to(device)

# 데이터셋과 데이터로더 생성
from torch.utils.data import DataLoader, TensorDataset

batch_size = 4096  # 배치 크기를 늘려서 GPU 활용을 극대화
dataset = TensorDataset(X_input_tensor, Y_true_tensor)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# KANLinear layer with spline basis
class KANLinear(nn.Module):
    def __init__(self, in_features, out_features, grid_size=10, degree=3):
        super(KANLinear, self).__init__()
        self.in_features = in_features  # Should be 1 for φ_i functions
        self.out_features = out_features
        self.degree = degree
        self.grid_size = grid_size

        # Knot vector (uniformly spaced)
        self.num_knots = grid_size + 2 * degree  # Corrected number of knots
        knots = np.linspace(-2 * np.pi, 2 * np.pi, self.num_knots)
        self.register_buffer('knots', torch.from_numpy(knots).float())

        # Number of basis functions
        self.n_basis = self.num_knots - degree - 1  # Corrected number of basis functions

        # Coefficients for each basis function
        # Since in_features is 1, we can simplify weights to a single nn.Parameter
        self.weights = nn.Parameter(torch.randn(self.n_basis, out_features))

    def forward(self, x):
        # x: [batch_size, 1]
        xi = x.squeeze(1)  # [batch_size]
        basis = self.bspline_basis(xi, self.knots, self.degree)  # [batch_size, n_basis]
        out = basis @ self.weights  # [batch_size, out_features]
        return out

    def bspline_basis(self, x, knots, degree):
        # x: [batch_size]
        # knots: [num_knots]
        x = x.unsqueeze(1)  # [batch_size, 1]

        # Number of basis functions
        n_basis = self.n_basis

        # Initialize degree 0 basis functions
        basis = []
        for i in range(n_basis + degree):
            cond = ((x >= knots[i]) & (x < knots[i + 1])).float()
            basis.append(cond)
        basis = torch.stack(basis, dim=2)  # [batch_size, 1, n_basis + degree]

        # Recursive computation of basis functions
        for k in range(1, degree + 1):
            new_basis = []
            for i in range(n_basis + degree - k):
                denom1 = knots[i + k] - knots[i]
                denom2 = knots[i + k + 1] - knots[i + 1]

                term1 = 0
                if denom1 != 0:
                    term1 = ((x - knots[i]) / denom1) * basis[:, 0, i]
                if denom2 != 0:
                    term2 = ((knots[i + k + 1] - x) / denom2) * basis[:, 0, i + 1]
                else:
                    term2 = 0

                new_basis.append(term1 + term2)
            basis = torch.stack(new_basis, dim=2)  # [batch_size, 1, n_basis + degree - k]

        return basis.squeeze(1)  # [batch_size, n_basis]

# KAN model definition
class KAN(nn.Module):
    def __init__(self, input_dim, hidden_size, grid_size=10, degree=3):
        super(KAN, self).__init__()
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        
        # 각 입력 차원에 대한 일변량 함수 φ_i
        self.phi_layers = nn.ModuleList([
            nn.Sequential(
                KANLinear(1, hidden_size, grid_size, degree),
                nn.SiLU(),
                nn.Linear(hidden_size, hidden_size),
                nn.SiLU()
            ) for _ in range(input_dim)
        ])

        # 최종 함수 ψ
        self.psi = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.SiLU(),
            nn.Linear(hidden_size, 1)
        )  
        

    def forward(self, x):
        # x: [batch_size, input_dim]
        outputs = []
        for i in range(self.input_dim):
            xi = x[:, i].unsqueeze(1)  # [batch_size, 1]
            phi_i = self.phi_layers[i](xi)  # [batch_size, hidden_size]
            outputs.append(phi_i)
        # 출력 합산
        s = sum(outputs)  # [batch_size, hidden_size]
        # 최종 함수 적용
        out = self.psi(s)  # [batch_size, 1]
        return out.squeeze()

# MLP 모델 정의 (KAN과 구조를 맞춤)
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_size, n_hidden_layers=3):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_size))
        layers.append(nn.SiLU())
        for _ in range(n_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.SiLU())
        layers.append(nn.Linear(hidden_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze()

# Saver 클래스 정의
class Saver:
    def __init__(self):
        self.outputs = []

    def save(self, epoch, Y_pred):
        self.outputs.append((epoch, Y_pred))

# 모델 초기화 및 장치로 이동
input_dim = 2
hidden_size = 16  # 히든 사이즈를 늘려서 복잡한 함수 근사에 적합하게 조정
n_hidden_layers = 4  # 히든 레이어 수를 늘림
grid_size = 10
degree = 3

kan_model = KAN(input_dim=input_dim, hidden_size=hidden_size, grid_size=grid_size, degree=degree).to(device)
mlp_model = MLP(input_dim=input_dim, hidden_size=hidden_size * 10, n_hidden_layers=n_hidden_layers).to(device)

# 모델 요약 출력
print("KAN 모델 요약:")
summary(kan_model, input_size=(input_dim,), device=str(device))

print("\nMLP 모델 요약:")
summary(mlp_model, input_size=(input_dim,), device=str(device))

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
kan_optimizer = optim.Adam(kan_model.parameters(), lr=0.001)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# AMP를 위한 GradScaler 초기화
kan_scaler = torch.amp.GradScaler('cuda')
mlp_scaler = torch.amp.GradScaler('cuda')

# 학습 루프
epochs = 100

# 에포크별 손실 저장용 리스트
kan_losses = []
mlp_losses = []

# 시각화를 위한 예측값 저장용 Saver 객체 생성
kan_saver = Saver()
mlp_saver = Saver()

# 시각화할 에포크 지정 (5개의 에포크로 줄임)
visualization_epochs = [19, 39, 59, 79, 99]  # 총 5개의 에포크

for epoch in range(epochs):
    kan_model.train()
    mlp_model.train()
    
    kan_epoch_loss = 0
    mlp_epoch_loss = 0
    
    for X_batch, Y_batch in loader:
        # 데이터를 장치로 이동
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        # 그래디언트 초기화
        kan_optimizer.zero_grad()
        mlp_optimizer.zero_grad()
        
        # AMP를 사용하여 순전파 및 역전파
        with torch.amp.autocast('cuda'):
            # KAN 모델 학습
            kan_outputs = kan_model(X_batch)
            kan_loss = criterion(kan_outputs, Y_batch)
        
        # 손실 스케일링 및 역전파
        kan_scaler.scale(kan_loss).backward()
        kan_scaler.step(kan_optimizer)
        kan_scaler.update()
        
        kan_epoch_loss += kan_loss.item() * X_batch.size(0)
        
        with torch.amp.autocast('cuda'):
            # MLP 모델 학습
            mlp_outputs = mlp_model(X_batch)
            mlp_loss = criterion(mlp_outputs, Y_batch)
        
        # 손실 스케일링 및 역전파
        mlp_scaler.scale(mlp_loss).backward()
        mlp_scaler.step(mlp_optimizer)
        mlp_scaler.update()
        
        mlp_epoch_loss += mlp_loss.item() * X_batch.size(0)
    
    # 에포크별 평균 손실 계산
    kan_epoch_loss /= len(dataset)
    mlp_epoch_loss /= len(dataset)
    
    # 손실 저장
    kan_losses.append(kan_epoch_loss)
    mlp_losses.append(mlp_epoch_loss)
    
    # 손실 출력
    print(f'Epoch {epoch+1}/{epochs}, KAN Loss: {kan_epoch_loss:.4f}, MLP Loss: {mlp_epoch_loss:.4f}')
    
    # 특정 에포크마다 예측값 저장
    if epoch in visualization_epochs:
        # 모델 평가 모드로 전환
        kan_model.eval()
        mlp_model.eval()
        
        with torch.no_grad():
            # 전체 데이터셋을 장치로 이동
            X_input_tensor_device = X_input_tensor.to(device)
            
            # AMP를 사용하여 예측
            with torch.amp.autocast('cuda'):
                kan_pred = kan_model(X_input_tensor_device)
                mlp_pred = mlp_model(X_input_tensor_device)
            
            # 예측 값 저장 (CPU로 이동)
            kan_saver.save(epoch, kan_pred.cpu().numpy())
            mlp_saver.save(epoch, mlp_pred.cpu().numpy())

# 실제 함수 값 (CPU로 이동)
Z_true = Y_true.reshape(X1.shape)

# 손실 곡선 시각화
plt.figure(figsize=(10,5))
plt.plot(range(1, epochs+1), kan_losses, label='KAN Loss')
plt.plot(range(1, epochs+1), mlp_losses, label='MLP Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

# 에포크별 결과 시각화 함수 정의
def plot_approximations(X1, X2, Y_true, kan_saver, mlp_saver):
    num_cols = 5  # 한 행에 보여줄 그래프 수
    num_rows = 3  # 실제 함수 + KAN + MLP

    fig = plt.figure(figsize=(4*num_cols, 4*num_rows))

    # 실제 함수 시각화 (첫 번째 행)
    for i in range(num_cols):
        ax = fig.add_subplot(num_rows, num_cols, i+1, projection='3d')
        if i == 0:
            ax.plot_surface(X1, X2, Y_true.reshape(X1.shape), cmap='viridis')
            ax.set_title('Actual Function')
            ax.set_xlabel('x1')
            ax.set_ylabel('x2')
            ax.set_zlabel('f(x1, x2)')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_zticks([])
        else:
            # 빈 플롯 생성
            ax.axis('off')

    # KAN 에포크별 근사 결과 시각화 (두 번째 행)
    for i, (epoch, Y_pred) in enumerate(kan_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'KAN Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    # MLP 에포크별 근사 결과 시각화 (세 번째 행)
    for i, (epoch, Y_pred) in enumerate(mlp_saver.outputs):
        ax = fig.add_subplot(num_rows, num_cols, 2*num_cols + i+1, projection='3d')
        ax.plot_surface(X1, X2, Y_pred.reshape(X1.shape), cmap='viridis')
        ax.set_title(f'MLP Epoch {epoch+1}')
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('Approximation')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    plt.tight_layout()
    plt.show()

# 전체 결과 시각화
plot_approximations(X1, X2, Z_true, kan_saver, mlp_saver)


Using device: cuda
KAN 모델 요약:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         KANLinear-1                [-1, 2, 16]               0
              SiLU-2                [-1, 2, 16]               0
            Linear-3                [-1, 2, 16]             272
              SiLU-4                [-1, 2, 16]               0
         KANLinear-5                [-1, 2, 16]               0
              SiLU-6                [-1, 2, 16]               0
            Linear-7                [-1, 2, 16]             272
              SiLU-8                [-1, 2, 16]               0
            Linear-9                [-1, 2, 16]             272
             SiLU-10                [-1, 2, 16]               0
           Linear-11                 [-1, 2, 1]              17
Total params: 833
Trainable params: 833
Non-trainable params: 0
----------------------------------------------------------------
Input si

/root/miniconda3/envs/sam_env2/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([4096])) that is different to the input size (torch.Size([4096, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/root/miniconda3/envs/sam_env2/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([3136])) that is different to the input size (torch.Size([3136, 3136])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, KAN Loss: 0.6809, MLP Loss: 0.4992
Epoch 2/100, KAN Loss: 0.6080, MLP Loss: 0.4920
Epoch 3/100, KAN Loss: 0.6134, MLP Loss: 0.4739
Epoch 4/100, KAN Loss: 0.6143, MLP Loss: 0.4150
Epoch 5/100, KAN Loss: 0.5603, MLP Loss: 0.3337
Epoch 6/100, KAN Loss: 0.5520, MLP Loss: 0.2470
Epoch 7/100, KAN Loss: 0.5293, MLP Loss: 0.1660
Epoch 8/100, KAN Loss: 0.5939, MLP Loss: 0.1036
Epoch 9/100, KAN Loss: 0.5221, MLP Loss: 0.0716
Epoch 10/100, KAN Loss: 0.5159, MLP Loss: 0.0558
Epoch 11/100, KAN Loss: 0.5282, MLP Loss: 0.0450
Epoch 12/100, KAN Loss: 0.5110, MLP Loss: 0.0363
Epoch 13/100, KAN Loss: 0.5164, MLP Loss: 0.0299
Epoch 14/100, KAN Loss: 0.5163, MLP Loss: 0.0243
Epoch 15/100, KAN Loss: 0.5188, MLP Loss: 0.0206
Epoch 16/100, KAN Loss: 0.5112, MLP Loss: 0.0180
Epoch 17/100, KAN Loss: 0.5218, MLP Loss: 0.0141
Epoch 18/100, KAN Loss: 0.5073, MLP Loss: 0.0121
Epoch 19/100, KAN Loss: 0.5082, MLP Loss: 0.0107
Epoch 20/100, KAN Loss: 0.5121, MLP Loss: 0.0101


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.96 GiB. GPU 0 has a total capacity of 79.15 GiB of which 1019.62 MiB is free. Process 31775 has 78.15 GiB memory in use. Of the allocated memory 77.57 GiB is allocated by PyTorch, and 77.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)